In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

data_train = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
data_test = pd.read_csv('testData.tsv', sep='\t', quoting=3)

y_train = data_train["sentiment"]  
print(data_train.shape)
print(data_test.shape)

(25000, 3)
(25000, 2)


In [3]:
data_train[:3]

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

def review_to_words(review_raw, stop_words):
    review_text = BeautifulSoup(review_raw).get_text()
        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    words = letters_only.lower().split()
    
    meaningful_words = [w for w in words if not w in stop_words]
    
    return(" ".join(meaningful_words))


print "Cleaning and parsing movie reviews...\n"      
feature_train = []
m_train = len(data_train.review)
for i in xrange(m_train):
    if( (i+1)%5000 == 0 ):
        print "Review train %d of %d\n" % ( i+1, m_train )    
    feature_train.append( review_to_words(data_train.review[i], stops))
feature_test = []
m_test = len(data_test.review)
for i in xrange(m_test):
    if( (i+1)%5000 == 0 ):
        print "Review test %d of %d\n" % ( i+1, m_test )    
    feature_test.append( review_to_words(data_test.review[i], stops))

Cleaning and parsing movie reviews...



F:\Anaconda\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file F:\Anaconda\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review train 5000 of 25000

Review train 10000 of 25000

Review train 15000 of 25000

Review train 20000 of 25000

Review train 25000 of 25000

Review test 5000 of 25000

Review test 10000 of 25000

Review test 15000 of 25000

Review test 20000 of 25000

Review test 25000 of 25000



In [5]:
len(feature_train)

25000

In [ ]:
from sklearn.pipeline import Pipeline

pipe_lr = Pipeline([('tfidf', TfidfVectorizer), ('lr', LogisticRegression)])
paramaters = {
    'tfidf__'
}

In [17]:
print 'vectorizing... ', 
vectorizer = TfidfVectorizer(analyzer='word', max_features=5000)
feature_all = feature_train + feature_test
vectorizer_fit = vectorizer.fit(feature_all)
X_train = vectorizer_fit.transform(feature_train).toarray()
X_test = vectorizer_fit.transform(feature_test).toarray()

vectorizing... 


In [18]:
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, 
                         C=1, fit_intercept=True, intercept_scaling=1.0, 
                         class_weight=None, random_state=1031, n_jobs=-1)
print "20 Fold CV Score: ", np.mean(cross_validation.cross_val_score(model, X_train, y_train, cv=20, scoring='roc_auc'))

20 Fold CV Score:  0.929038592


In [21]:
print "Retrain on all training data, predicting test labels...\n"
model.fit(X_train, y_train)
result = model.predict_proba(X_test)[:,1]
output = pd.DataFrame( data={"id": data_test.id, "sentiment": result} )

# Use pandas to write the comma-separated output file
output.to_csv('logistic.csv', index=False, quoting=3)
print "Wrote results to logistic.csv"

Retrain on all training data, predicting test labels...

Wrote results to Bag_of_Words_model.csv
